# fshafas Examples

In [ ]:
import pandas as pd
from statistics import mean
from functools import reduce
from ipyleaflet import Map, Marker, Icon, MarkerCluster
from fshafas.fable_modules.fs_hafas_profiles_python.db.profile import profile as db_profile
from fshafas.fable_modules.fs_hafas_python.lib.transformations import (
    Default_LocationsOptions, Default_JourneysOptions, Default_RadarOptions)
from fshafas.fable_modules.fs_hafas_python.types_hafas_client import (
    Station, Stop, Location, Leg, Journeys, Journey, BoundingBox, Line, Movement)
from fshafas.hafas_client import HafasClient
from util import (to_locations, to_name, to_dicts, to_dicts_of_legs, transform_name_of_line, flatten_location, centroid, to_bounding_box)


## Locations

Retrieve location data of Hannover.

In [ ]:
with HafasClient(db_profile) as client:
    stops = await client.locations("Hannover", Default_LocationsOptions)
    locations = to_locations(stops)


In [ ]:
pd.DataFrame(to_dicts(locations), columns=["id", "name", "longitude", "latitude"])

## Journeys

Retrieve current journeys from Hamm to Hamburg.

In [ ]:
with HafasClient(db_profile) as client:
    journeys_result: Journeys = await client.journeys("Hamm", "Hamburg", Default_JourneysOptions)
    journeys = journeys_result.journeys

In [ ]:
legs, _ = reduce(lambda acc, journey: [acc[0] +
                                        to_dicts_of_legs(journey, acc[1]), acc[1] + 1], journeys, ([], 0))
tuples = map(lambda l: (l["id"], l["price"], l["origin"]), legs)
index = pd.MultiIndex.from_tuples(tuples, names=["id", "price", "origin"])

pd.DataFrame(legs, columns=["destination",
             "departure", "arrival"], index=index)


## Radar

Retrieve all trains currently in an area between Münster and Hannover.

In [ ]:
with HafasClient(db_profile) as client:
    stops_1 = await client.locations("Münster", Default_LocationsOptions)

    stops_2 = await client.locations("Hannover", Default_LocationsOptions)

rect = to_bounding_box(stops_1, stops_2)

with HafasClient(db_profile) as client:
    opt = Default_RadarOptions
    opt.duration = 2400
    opt.frames = 10
    opt.products = client.productsOfMode(db_profile, "train")
    movements = await client.radar(rect, opt)


In [ ]:
pd.DataFrame(to_dicts(movements, transform=transform_name_of_line,
             flatten=flatten_location), columns=["trip_id", "line", "longitude", "latitude"])


Display train positions on a map.

In [ ]:
m = Map(center=centroid(movements, rect), zoom=8)

icon_url = 'https://raw.githubusercontent.com/Leaflet/Leaflet/main/dist/images/marker-icon.png'
icon = Icon(icon_url=icon_url, icon_size=[25, 40], icon_anchor=[22, 94])

markers = [Marker(location=(m.location.latitude, m.location.longitude), title=to_name(m.line),
                  icon=icon, draggable=False) for m in movements]

marker_cluster = MarkerCluster(markers=markers)

m.add_layer(marker_cluster)

m
